In [ ]:

from sklearn.ensemble import RandomForestClassifier
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing
import os
print(os.listdir("../input"))


### Loading datasets

In [ ]:
%%time
event_type=pd.read_csv("../input/event_type.csv",error_bad_lines=False)
train = pd.read_csv("../input/train.csv")
severity_type = pd.read_csv("../input/severity_type.csv")
log_feature = pd.read_csv("../input/log_feature.csv")
test = pd.read_csv("../input/test.csv")
resource_type = pd.read_csv("../input/resource_type.csv",error_bad_lines=False)
sample_submission = pd.read_csv("../input/sample_submission.csv")

In [ ]:
print("test",test.shape)
print("train",train.shape)

### Input datasets heads

In [ ]:
print('test',test.head())
print('train',train.head(4))
print('sample_submission',sample_submission.head())
print('event_type',event_type.shape,event_type.head(2))
print('severity_type',severity_type.shape,severity_type.head(2))
print('log_feature',log_feature.shape,log_feature.head(2))
print('resource_type',resource_type.shape,resource_type.head(2))

In [ ]:
train.head()

## ** Data conversion**

In [ ]:
event_type['id']=pd.to_numeric(event_type['id'],errors='coerce')
#converting object datatype into numeric

In [ ]:
event_type.dtypes

# **Training Preprocessing**

### **Merging**

In [ ]:
train_eve_mer=train.merge(event_type.drop_duplicates(subset=['id']),on='id')

In [ ]:
train_merge2=train_eve_mer.merge(severity_type.drop_duplicates(subset=['id']),how='left',on='id')

In [ ]:
train_merge3=pd.merge(train_merge2,log_feature.drop_duplicates(subset=['id']),how='left',on='id')

In [ ]:
train_merge4=pd.merge(train_merge3,resource_type.drop_duplicates(subset=['id']),how='left',on='id')

#### **Merged Training data**

In [ ]:
train_merge4.head()

In [ ]:
train_merge4.dtypes

### **Checking for missing values**

In [ ]:
train_merge4.isnull().sum()

### **Finding categorical columns**

In [ ]:
cat_col=list(set(train_merge4.columns)-set(train_merge4._get_numeric_data().columns))

### **Categorical conversion**

In [ ]:
for i in range(len(cat_col)):
    train_merge4[cat_col[i]]=train_merge4[cat_col[i]].astype('category')

    

### **Label encoding**

In [ ]:
le=preprocessing.LabelEncoder()
for i in range(len(cat_col)):
    train_merge4[cat_col[i]]=le.fit_transform(train_merge4[cat_col[i]])

In [ ]:
train_merge4.drop(['id'],axis=1,inplace=True)

In [ ]:
target=train_merge4[['fault_severity']]

In [ ]:
train_merge4.drop(['fault_severity'],axis=1,inplace=True)

## modeling


In [ ]:
rfc=RandomForestClassifier()

In [ ]:
rfc.fit(train_merge4,target)

##Tried XGBOOST classifier ,throwing error
from sklearn.model_selection import train_test_split
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(train_merge4, target, test_size=test_size, random_state=5)

import xgboost as xgb
dtrain = xgb.DMatrix(data=X_train,label=y_train)

params = {'learning_rate':0.1,
        'n_estimators':500,
        'num_class':3,
        'eval_metric':['mlogloss'],
        'missing':-1,
        'nthread':4}

num_round = 10000
bst = xgb.train(params, dtrain, num_round)

dtest = xgb.DMatrix(data=train_merge4,label=target)

n_classes =3
preds = bst.predict(dtest,n_classes)

pd.DataFrame(preds).head()

best_preds

import numpy as np
best_preds = np.asarray([np.argmax(line) for line in preds])
#Calcul de précision
from sklearn.metrics import accuracy_score
print(accuracy_score(best_preds, target))

In [ ]:
train_merge4.head()

In [ ]:
test.head()

## **TEST data preparation**

In [ ]:
test.head()

In [ ]:
test.shape

## ** Test data merging**

In [ ]:
test_merge1=pd.merge(test,event_type.drop_duplicates(subset=['id']),how='left',on='id')

In [ ]:
test_merge2=pd.merge(test_merge1,severity_type.drop_duplicates(subset=['id']),how='left',on='id')

In [ ]:
test_merge3=pd.merge(test_merge2,log_feature.drop_duplicates(subset=['id']),how='left',on='id')

In [ ]:
test_merge4=pd.merge(test_merge3,resource_type.drop_duplicates(subset=['id']),how='left',on='id')

In [ ]:
test_merge4.head()

In [ ]:
cat_col

In [ ]:
test_merge4.dtypes

### **Categorical conversion **

In [ ]:
for i in range(len(cat_col)):
    test_merge4[cat_col[i]]=test_merge4[cat_col[i]].astype('category')


### **Label encoding**

In [ ]:
le2=preprocessing.LabelEncoder()
for i in range(len(cat_col)):
    test_merge4[cat_col[i]]=le2.fit_transform(test_merge4[cat_col[i]])

In [ ]:
test_merge4.dtypes

#### **Droping unique column**

In [ ]:
test_merge4.drop(['id'],axis=1,inplace=True)

In [ ]:
train_merge4.columns

In [ ]:
test_merge4.columns

#### **Prediction **

In [ ]:
predict_test=rfc.predict_proba(test_merge4)

In [ ]:
pred_df=pd.DataFrame(predict_test,columns=['predict_0', 'predict_1', 'predict_2'])

In [ ]:
submission=pd.concat([test[['id']],pred_df],axis=1)

### **Submission file generation**

In [ ]:
submission.to_csv('sub.csv',index=False,header=True)